In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/raw/train.csv')
data.head()

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0


## Соотношение классов

In [3]:
data['is_duplicate'].value_counts()

0    494161
1      3658
Name: is_duplicate, dtype: int64

Видим очень сильный дисбаланс в сторону класса 0 (99.3%)

### Класс 1

In [4]:
data[data['is_duplicate']==1]

,pair_id,name_1,name_2,is_duplicate
161,162,JX Nippon Oil & Gas Exploration (Brasil) Ltda,JX Nippon Oil & Gas Exploration Technical Serv...,1
603,604,Pirelli Neumaticos S.A.I.C.,"Pirelli Tyre Co., Ltd.",1
835,836,Brenntag Australia (Pty) Ltd.,Brenntag Group,1
1328,1329,"PAUL BAUDER GMBH & CO KG, BOCHUM PLANT",Paul Bauder ag,1
1562,1563,TOTAL CESKA REPUBLIKA s.r.o.,TOTAL FRANCE (ARNAY LE DUC),1
...,...,...,...,...
496574,496575,"Bridgestone （Huizhou）Synthetic Rubber Co., Ltd.","Bridgestone India Pvt., Ltd.",1
496760,496761,Arlanxeo International Group,Arlanxeo Corp.,1
497015,497016,Brenntag Peru S.A.C.,Brenntag Chile Comercial E Industrial Ltda,1
497083,497084,Dow Chemical International Private Ltd.,Dow Chemical Pacific,1


### Сколько всего уникальных названий компаний

In [5]:
unique_companies = pd.concat([data['name_1'], data['name_2']]).unique()
len(unique_companies)

18022

### Сколько из них встречаюся в классе 1:

In [6]:
name_1_duplicates = data.loc[data['is_duplicate'] == 1, 'name_1']
name_2_duplicates = data.loc[data['is_duplicate'] == 1, 'name_2']
unique_companies_class_1 = pd.concat([name_1_duplicates, name_2_duplicates]).unique()
len(unique_companies_class_1)

1394

## Анализ на уровне символов

In [7]:
symbols = set()
for name in unique_companies:
    symbols.update(name.lower())

In [8]:
symbols

{' ',
 '"',
 '#',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '\xa0',
 '«',
 '±',
 '»',
 '¿',
 'á',
 'ã',
 'ç',
 'è',
 'é',
 'í',
 'ñ',
 'ó',
 'õ',
 'ö',
 'ú',
 'ü',
 'ę',
 'ł',
 'ő',
 'ş',
 'ű',
 '̇',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ы',
 'ь',
 'э',
 'ю',
 'я',
 'ё',
 'ر',
 'س',
 'ف',
 'ك',
 'م',
 'و',
 'ي',
 '\u0e00',
 '‘',
 '’',
 '\u3000',
 '。',
 '上',
 '东',
 '京',
 '份',
 '会',
 '公',
 '北',
 '双',
 '司',
 '团',
 '式',
 '彤',
 '技',
 '料',
 '新',
 '方',
 '日',
 '有',
 '术',
 '材',
 '株',
 '水',
 '海',
 '社',
 '程',
 '股',
 '虹',
 '防',
 '限',
 '集',
 '雨',
 '﹠',
 '＆',
 '（',
 '）',
 '，'}

Помимо англ и русских букв встречается много знаков препинания, цифры + китайский и арабский(?) алфавит. Посмотрим насколько это частое явление.

In [10]:
# Арабский символы

companies_with_arabic_symbols = set()
for s in ['ر', 'س', 'ف', 'ك', 'م', 'و', 'ي']:
    companies_with_arabic_symbols.update([name for name in unique_companies if s in name])
companies_with_arabic_symbols

{'Servicom S.A. (BVMT:SERVI) - سرفيكوم'}

Всего 1 компания с арабскими символами в названии

In [11]:
# Китайские иероглифы

companies_with_chinese_symbols = set()
for s in ['上', '东', '京', '份', '会', '公', '北', '双', '司', '团', '式', '彤', '技', '料', '新', '方', '日', '有', '术', '材', '株', '水', '海', '社', '程', '股', '虹', '防', '限', '集', '雨']:
    companies_with_chinese_symbols.update([name for name in unique_companies if s in name])
companies_with_chinese_symbols

{'Beijing Oriental Yuhong Waterproof Technology Co., Ltd. (SZSE:002271) - 北京东方雨虹防水技术股份有限公司',
 'Jsrtrading（上海）Co. Ltd.',
 'Red Avenue New Materials Group Co., Ltd (彤程新材料集团股份有限公司)',
 'Sojitz Corporation (TSE:2768) - 双日株式会社'}

Всего 4 компании с китайскими иероглифами в названии

In [12]:
companies_with_numbers = set()
for s in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
    companies_with_numbers.update([name for name in unique_companies if s in name])
companies_with_numbers

{'1 4103412224 Tel Ex 1 410341 23',
 '1 7044245522 Tel Ex 1 704424 56',
 '1 8168912400 Tel Ex 1 816891 76',
 '1 Dhl Global Forwarding',
 '1 St Notify Party',
 '1 St World Medical Supplies Solution Inc.',
 '10841 Ambassador Drive',
 '123 E Latin America S De Rl De Cv',
 '128 Yeouidaero Youngdeungpo Gu',
 '14 Th Fl. Guomao Building Hubin',
 '2 Nd Bulkhaul (Usa) Inc.',
 '2 Nd Norify Party Goodyear Canada Inc.',
 '2 Wedmore Close',
 '21 St Century Textiles Ltd.',
 '210 Brands Inc.',
 '210 Brands Inc. Dba Canterbury Usa',
 '210 Brands Inc./Canterbury Offi',
 '2K Polymer Systems Ltd ',
 '3 A Pakistan',
 '3 Ds Traders',
 '3 K Treads (Pvt) Ltd.',
 '3 M Brockville Tape',
 '3 M Do Brasil Ltda',
 '3 M India Ltd.',
 '3 M Manaus Industria De Produtos Quimicos Ltda',
 '3 M United Kingdom Plc 3 M Center',
 '3 Pl Guys',
 '3 Plus Logistics (Atlanta)',
 '3 Plus Logistics Co.',
 '3 Rd Floor Gate 4 Nanpu Building Tia',
 '3 S Fabrications (Pvt) Ltd.',
 '31 Mcu Movement Control Unit',
 '3475 Piedmont Rd. Ne

Кажется, что цифры в названиях чаще являются шумом (возможно их стоит тоже почистить)